In [1]:
%matplotlib inline
%cd ../

G:\Workspace\SpatialPyramidMatchExp


In [2]:
from pprint import pprint
import os
from os import path as ospath
from functools import reduce

import matplotlib.pyplot as plt
import numpy as np

from cvtools import io
from cvtools import spm

In [3]:
train_data_path = 'dataset_full/training'
test_data_path = 'dataset_full/testing'
save_data_dir = 'data'

In [4]:
# 导入训练数据数据
train_data = tuple(io.get_images_name(train_data_path,recursive=True))
train_images = io.load_image2ndarray(train_data)
train_labels = io.get_image_label_in_filename(train_data)

In [5]:
path_prefix = save_data_dir
filename_fmt = 'spm_%s_%d_%d.pkl'# 第一个是使用的核函数，第二个是训练集大小,第三个是层数
# 导入或构造SPM模型
def get_spm_model(svm_kernel,filename = 'spm.pkl',level=2):
    spm_model_path = ospath.join(path_prefix,filename)
    spm_model = None
    if ospath.exists(spm_model_path):
        print("SPM Model of %s is existed,loading %s ..."  % (svm_kernel,filename))
        spm_model = io.load_data(path_prefix=path_prefix, filename=filename)
        print("loading done!")
    else:
        print("Training SPM Model of %s ...." % (svm_kernel,))
        spm_model = spm.SpatialPyramidMatch(train_images,train_labels,svm_kernel=svm_kernel,pyramid_level=level,show_msg=True) 
        spm_model_path = io.save_data(spm_model, path_prefix=path_prefix, filename=filename)
        print("SPM Model Save Path:",spm_model_path)
    return spm_model

In [6]:
# 性能评估方法
def classification(spm_model,test_data,test_images,test_labels,level,show_all=False):
    answer = spm_model.predict_images(test_images)
    output_labels = [ spm_model.get_label(ans) for ans in answer ]
    correct = len(tuple(filter(lambda x:x[0]==x[1],zip(test_labels,output_labels))))
    accuracy = correct / len(test_labels)
    print("Kernel: %12s\taccuracy:%.2f%%(%d/%d)" % (spm_model.svm_kernel,accuracy*100,correct,len(test_labels)))
    if show_all:
        print("target\t\t\t\toutput")
        print("**************************************************************")
        for path,target,output in zip(test_data,test_labels,output_labels):
            if target != output:
                print("%s\t\t\t\t%s" % (target,output))

In [7]:
# 使用不同的核函数构造模型
spm_models = {}
svm_kernel = ('linear','poly','rbf','sigmoid','hi')
plevel = (0,1,2,3,4)
train_data_size = len(train_images)
def run_and_get_time(kernel,train_data_size,level):
    spm_models[level][kernel] = get_spm_model(kernel,filename,level)

for level in plevel:
    spm_models[level] = {}
    for kernel in svm_kernel:
        filename = filename_fmt % (kernel,train_data_size,level)
        run_and_get_time(kernel,train_data_size,level)

Training SPM Model of linear ....
calculate sift is running ...
generate vocabulary dictionary is running ...
init label set and label list is running ...
build pyramid is running ...
train classificator is running ...
SPM Model Save Path: data\spm_linear_240_0.pkl
Training SPM Model of poly ....
calculate sift is running ...
generate vocabulary dictionary is running ...
init label set and label list is running ...
build pyramid is running ...
train classificator is running ...
SPM Model Save Path: data\spm_poly_240_0.pkl
Training SPM Model of rbf ....
calculate sift is running ...
generate vocabulary dictionary is running ...
init label set and label list is running ...
build pyramid is running ...
train classificator is running ...
SPM Model Save Path: data\spm_rbf_240_0.pkl
Training SPM Model of sigmoid ....
calculate sift is running ...
generate vocabulary dictionary is running ...
init label set and label list is running ...
build pyramid is running ...
train classificator is runn

In [8]:
# 导入测试数据
test_data = tuple(io.get_images_name(test_data_path,recursive=True))
test_images = io.load_image2ndarray(test_data)
test_labels = io.get_image_label_in_filename(test_data)

In [9]:
# 测试
print("testing ...")
for level,serial in spm_models.items():
    print('\n')
    print('Level:',level)
    for kernel,model in serial.items():
        classification(model,test_data,test_images,test_labels,level,show_all=False)

testing ...


Level: 0
Kernel:       linear	accuracy:42.50%(51/120)
Kernel:         poly	accuracy:20.83%(25/120)
Kernel:          rbf	accuracy:37.50%(45/120)
Kernel:      sigmoid	accuracy:39.17%(47/120)
Kernel:           hi	accuracy:53.33%(64/120)


Level: 1
Kernel:       linear	accuracy:41.67%(50/120)
Kernel:         poly	accuracy:16.67%(20/120)
Kernel:          rbf	accuracy:45.83%(55/120)
Kernel:      sigmoid	accuracy:51.67%(62/120)
Kernel:           hi	accuracy:66.67%(80/120)


Level: 2
Kernel:       linear	accuracy:40.83%(49/120)
Kernel:         poly	accuracy:59.17%(71/120)
Kernel:          rbf	accuracy:46.67%(56/120)
Kernel:      sigmoid	accuracy:46.67%(56/120)
Kernel:           hi	accuracy:66.67%(80/120)


Level: 3
Kernel:       linear	accuracy:50.00%(60/120)
Kernel:         poly	accuracy:51.67%(62/120)
Kernel:          rbf	accuracy:20.00%(24/120)
Kernel:      sigmoid	accuracy:40.83%(49/120)
Kernel:           hi	accuracy:65.00%(78/120)


Level: 4
Kernel:       linear	accuracy:45.